In [ ]:
## For colab runs
# from google.colab import drive
# drive.mount('/content/gdrive')

# !mkdir -p data
# !cp /content/gdrive/My\ Drive/uploads/a9a.2 ./data # load this data to the specified dir on your drive first

# DGM Minimal Environment 

In [ ]:
import torch
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_svmlight_file
from src.objectives import ( 
    Objective,
    LeastSquares, LogRegression,
    StochLeastSquares, StochLogRegression)
from src.methods import (
    EXTRA, DIGing, MDAccGD,
    DAccGD, Mudag, DSGD)
from src.utils import PythonGraph

# from src.sparse import objectives as obj
warnings.simplefilter('ignore')

In [ ]:
num_nodes = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
A, b = load_svmlight_file('data/a9a')
A = torch.Tensor(A.todense()).to(device)
b = torch.Tensor(b).to(device)

In [ ]:
# For simulating a graph evolution,
# only graphs like 'erdos_renyi' is appropriate

p = .65
graph = 'random_geometric'
# graph = 'erdos_renyi'
# graph = 'path'
# graph = 'cycle'
# graph = 'complete'
sigma = 1e-2

F = LogRegression(A, b, num_nodes, sigma)
Fs = StochLogRegression(A, b, num_nodes, sigma)
X0 = torch.zeros(num_nodes, A.size(1)).to(device)
gen = lambda : PythonGraph(F, graph, p).gen()[1]
# gen = lambda: consensus_variation(F, p)

opts = []
opts.append(
    EXTRA(F, gen, eta=12.))

opts.append(
    DIGing(F, gen, eta=12.))

consensus_iters = 10
opts.append(
    DAccGD(F, gen, L=.035, mu=sigma, con_iters=consensus_iters))

opts.append(
    Mudag(F, gen, L=.035, mu=sigma, M=.2, kappa_g=1e8))

opts.append(
    MDAccGD(F, gen, L=.035, mu=sigma, M=.2, kappa_g=1e8))

# M has almost no effect on the convergence

In [ ]:
%%time
n_iters = 100
for opt in opts:
    opt.run(X0, n_iters=n_iters//opt.con_iters);
    opt.logs['nmix'] = np.array(opt.logs['i']) * opt.con_iters

In [ ]:
with open('./logreg_solutions/a9a/regcoef=0.00e+00', 'rb') as file:
    f_star = pickle.load(file)['func_star']

In [ ]:
# you can manually build an array which will define 
plt.figure(figsize=(8, 8))
for i, opt in enumerate(opts):
    lbl = opt.__class__.__name__
    plt.plot(
        opt.logs['nmix'], opt.logs['fn'] - f_star,
        marker=i+5, label=lbl, markevery=n_iters//(5+i*3))
    
plt.title('Optimization Functional Value over Iteration Number', size=20)
plt.ylabel(r'$f(\overline{x}_k) - f^*$', size=20)
plt.xlabel('communication steps', size=20)
plt.yscale('log')
plt.grid()
plt.legend();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_residual.png')

In [ ]:
plt.figure(figsize=(8, 8))
for i, opt in enumerate(opts):
    lbl = opt.__class__.__name__
    plt.plot(
        opt.logs['nmix'], opt.logs['dist2con'],
        marker=i+5, label=lbl, markevery=n_iters//(5+i*3))

plt.ylabel(r'$||(I-\frac{1}{n}11^T)X||^2$', size=20)
plt.xlabel('communication steps', size=20)
plt.yscale('log')
plt.legend()
plt.grid();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_consensus.png')